# 4. 판다스 데이터 쿼리 처리 

### 간단한 데이터베이스 사용

In [1]:
import sqlite3

### 판다스 모듈 사용

In [2]:
import pandas as pd

## 4-1 기본 테이블 확인하기 

### 데이터베이스 경로 지정

In [3]:
path = "../data/beat.db"

### 데이터베이스 컨넥션 생성 

In [4]:
con = sqlite3.connect(path)

### 데이터베이스 내부의 테이블 확인 

In [5]:
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", con)

,name
0,Band


### 테이블 데이터 조회하기 

In [6]:
pd.read_sql_query("SELECT * FROM Band;", con)

,id,fname,lname,birthyear
0,0,Paul,McCartney,1942
1,1,John,Lennon,1940


## 4-2 다양한 테이블 확인하기 

### 데이터베이스 확인 

In [7]:
path1 = "../data/chinook.db"

### 데이터베이스 컨넥션

In [8]:
con1 = sqlite3.connect(path1)

### 데이터베이스 내의 테이블 확인 

In [9]:
df_table = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", con1)

In [10]:
df_table

,name
0,albums
1,sqlite_sequence
2,artists
3,customers
4,employees
5,genres
6,invoices
7,invoice_items
8,media_types
9,playlists


### 테이블 내의 스키마 정보 확인 

In [11]:
for x in df_table['name'] :
    df = pd.read_sql_query(f"SELECT * FROM {x};", con1)
    print(df.info())
    print("="*40)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347 entries, 0 to 346
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   AlbumId   347 non-null    int64 
 1   Title     347 non-null    object
 2   ArtistId  347 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 8.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    10 non-null     object
 1   seq     10 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 288.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275 entries, 0 to 274
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ArtistId  275 non-null    int64 
 1   Name      275 non-null    object
dtypes: int64(1), object(1)
memory usage: 4.4+ KB
None
<class 'pandas.core.frame.DataFrame

### 테이블 조회 

In [12]:
df_albums = pd.read_sql_query("SELECT * FROM albums;", con1)

In [13]:
df_albums.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347 entries, 0 to 346
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   AlbumId   347 non-null    int64 
 1   Title     347 non-null    object
 2   ArtistId  347 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 8.3+ KB


### 특정 건수를 읽어오기  : sql

In [14]:
sql_1 = "select * from albums limit 5"

In [15]:
pd.read_sql_query(sql_1, con1)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


### 특정 건수를 읽어오기  : pandas 메서드 사용

In [16]:
df_albums.head()

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


### 마지막 부분 조회하기 

In [17]:
sql_2 = "select * from albums order by AlbumId Desc limit 5"

In [18]:
pd.read_sql_query(sql_2, con1)

,AlbumId,Title,ArtistId
0,347,Koyaanisqatsi (Soundtrack from the Motion Pict...,275
1,346,Mozart: Chamber Music,274
2,345,Monteverdi: L'Orfeo,273
3,344,Schubert: The Late String Quartets & String Qu...,272
4,343,Respighi:Pines of Rome,226


In [19]:
df_albums.tail()

,AlbumId,Title,ArtistId
342,343,Respighi:Pines of Rome,226
343,344,Schubert: The Late String Quartets & String Qu...,272
344,345,Monteverdi: L'Orfeo,273
345,346,Mozart: Chamber Music,274
346,347,Koyaanisqatsi (Soundtrack from the Motion Pict...,275


### 다른 테이블 조회

In [20]:
df_artists = pd.read_sql_query("SELECT * FROM artists;", con1)

In [21]:
df_artists.head()

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains


## 4-3 데이터 조인 처리 

-  두 테이블을 결합해서 하나의 테이블로 처리
- 특정 키 값이 같아야 한다. 

### 조인쿼리 만들기 : from과 where절 사용 처리

In [22]:
sql_j01 = """
select AlbumId,Title, al.ArtistId, Name 
from albums as al, artists as ar 
where al.ArtistId = ar.ArtistId;
"""

### 쿼리 실행하기

In [23]:
df_join = pd.read_sql_query(sql_j01, con1)

In [24]:
df_join.shape

(347, 4)

In [25]:
df_join.head()

,AlbumId,Title,ArtistId,Name
0,1,For Those About To Rock We Salute You,1,AC/DC
1,2,Balls to the Wall,2,Accept
2,3,Restless and Wild,2,Accept
3,4,Let There Be Rock,1,AC/DC
4,5,Big Ones,3,Aerosmith


### 조인하기 : join 쿼리 사용하기

In [26]:
sql_j02 = """
select AlbumId,Title, al.ArtistId, Name 
from albums as al
inner join artists as ar
on al.ArtistId = ar.ArtistId;
"""

In [27]:
df_join_02 = pd.read_sql_query(sql_j02, con1)

In [28]:
df_join_02.head()

,AlbumId,Title,ArtistId,Name
0,1,For Those About To Rock We Salute You,1,AC/DC
1,2,Balls to the Wall,2,Accept
2,3,Restless and Wild,2,Accept
3,4,Let There Be Rock,1,AC/DC
4,5,Big Ones,3,Aerosmith


## 4-4 서브쿼리 사용하기

### 위드구분으로 서브퀄를 만들고 사용하기

In [29]:
sql_with = """
With cte as
(SELECT * from Artists )
Select name from cte
"""

In [30]:
pd.read_sql_query(sql_with, con1)

,Name
0,AC/DC
1,Accept
2,Aerosmith
3,Alanis Morissette
4,Alice In Chains
...,...
270,"Mela Tenenbaum, Pro Musica Prague & Richard Kapp"
271,Emerson String Quartet
272,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
273,Nash Ensemble


In [31]:
### 서브 쿼리를 만들어서 처리하기 

In [32]:
sql_sub = """
SELECT * FROM
(SELECT * FROM Artists) AS Art 
ORDER BY 1 DESC
"""

In [33]:
pd.read_sql_query(sql_sub, con1)

,ArtistId,Name
0,275,Philip Glass Ensemble
1,274,Nash Ensemble
2,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
3,272,Emerson String Quartet
4,271,"Mela Tenenbaum, Pro Musica Prague & Richard Kapp"
...,...,...
270,5,Alice In Chains
271,4,Alanis Morissette
272,3,Aerosmith
273,2,Accept
